# Preparing the dataset for finetuning

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
raw_datasets["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [15]:
from transformers import AutoTokenizer

def tokenize_function(example):   
    checkpoint = "bert-base-cased" 
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    return tokenizer(
        example["text"], 
        padding="max_length",
        truncation=True,
        max_length=128
    )
tokenized_dataset = raw_datasets.map(tokenize_function, batched=True)    


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [16]:
small_train_ds = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_ds = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))
full_train_ds = tokenized_dataset["train"]
full_eval_ds = tokenized_dataset["test"]

# Finetuning in Pytorch with the trainer API

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

checkpoint = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np

training_args = TrainingArguments(
    "test_trainer",
    #ealuation_strategy="epoch"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_ds,
    eval_dataset=small_eval_ds
)